In [1]:
import re
import requests
import pandas as pd

In [3]:
def generate_category(category ):
    """
    format a category for insertion into a wikipedia api call
    """
    category = re.sub('\s','+', category)
    return category

In [4]:
generate_category('machine learning')

'machine+learning'

In [16]:
def generate_query(category):
    """
    format an api call for requests
    """
    query = """
            http://en.wikipedia.org/w/api.php?
            action=query&
            format=json&
            list=categorymembers&
            cmtitle=Category:{}&
            cmlimit=max
            """.format(generate_category(category))
    query = re.sub('\s','', query)
    return query

In [17]:
generate_query('machine learning')

'http://en.wikipedia.org/w/api.php?action=query&format=json&list=categorymembers&cmtitle=Category:machine+learning&cmlimit=max'

In [18]:
def execute_category_query(category):
    '''
    Executes a category qeury and returns a 
    DataFrame of the category members
    '''
    
    r = requests.get(generate_query(category))
    response = r.json()
    return pd.DataFrame(response['query']['categorymembers'])
    

In [25]:
test = execute_category_query('machine learning')

In [27]:
test

{'batchcomplete': '',
 'limits': {'categorymembers': 500},
 'query': {'categorymembers': [{'ns': 0,
    'pageid': 43385931,
    'title': 'Data exploration'},
   {'ns': 0,
    'pageid': 49082762,
    'title': 'List of datasets for machine learning research'},
   {'ns': 0, 'pageid': 233488, 'title': 'Machine learning'},
   {'ns': 0, 'pageid': 53587467, 'title': 'Outline of machine learning'},
   {'ns': 0, 'pageid': 53198248, 'title': 'Singular statistical model'},
   {'ns': 0, 'pageid': 3771060, 'title': 'Accuracy paradox'},
   {'ns': 0, 'pageid': 43808044, 'title': 'Action model learning'},
   {'ns': 0,
    'pageid': 28801798,
    'title': 'Active learning (machine learning)'},
   {'ns': 0, 'pageid': 45049676, 'title': 'Adversarial machine learning'},
   {'ns': 0, 'pageid': 52642349, 'title': 'AIVA'},
   {'ns': 0, 'pageid': 30511763, 'title': 'AIXI'},
   {'ns': 0, 'pageid': 50773876, 'title': 'Algorithm Selection'},
   {'ns': 0, 'pageid': 20890511, 'title': 'Algorithmic inference'},
   

In [29]:
category_mask = test['title'].str.contains('Category:')

KeyError: 'title'

In [31]:
def remove_category(category):
    category = re.sub('Category:','',category)
    return category

In [32]:
categories_to_query = test[category_mask]['title'].apply()

NameError: name 'category_mask' is not defined

In [22]:
categories_to_query = test[category_mask]['title'].apply(remove_category).tolist()

NameError: name 'test' is not defined

In [30]:
for category in categories_to_query:
    subcat_df_list.append(execute_ctegory_query(category))

NameError: name 'categories_to_query' is not defined

In [ ]:
len(subcat_df_list)

In [33]:
def get_all_pages_rec(category):
    category_df = execute_category_query(category)
    pages_list = []
    category_mask = category_df['title'].str.contains('Category:')
    pages_df = category_df[~category_mask]
    pages_list.append(pages_df)
    categories = category_df[category_mask]['title']\
                            .str.replace('Category:','').tolist()
    if len(categories) > 0:
        for cat in categories:
            pages_list.append(get_all_pages_rec(cat))
    
    pages_df = pd.concat(pages_list)
    pages_df.reset_index()
    return pages_df